In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

In [3]:
#options = EdgeOptions()
driver = webdriver.Edge()
driver.maximize_window()
data = pd.DataFrame()

driver.get("https://www.forbes.com/money/")
close = driver.find_element(By.XPATH, value="/html/body/div[8]/div/button")
close.click()
delay = 10 # seconds

count = 1   
#change 200 to other number if needs, as that is indicator for extract more news or less news depend on how many header an desc you look for.

for a in range(1,48):
    # Output to terminal repetitively over the same line using '\r'.
    sys.stdout.write("\rClick More Button Progress: {0}".format(a))
    sys.stdout.flush()
    more_articles = driver.find_element(By.XPATH, value="//button[contains(text(),'More Articles')]")
    more_articles.click()
    wait = WebDriverWait(driver, 10)
    more = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'More Articles')]")))
    driver.implicitly_wait(0.5)
    
for i in range(1,100):
    sys.stdout.write("\rItem: {0}".format(count))
    sys.stdout.flush()
    try:
        #print('get value {0}'.format(count))
        header = driver.find_element(By.XPATH, value="//*[@id=\"row-2\"]/div/div/div/div[1]/div/div/article[{0}]/div[1]/h3/a".format(count))
        desc = driver.find_element(By.XPATH, value="//*[@id=\"row-2\"]/div/div/div/div[1]/div/div/article[{0}]/div[1]/div[2]".format(count)) 
        author_label = driver.find_element(By.XPATH, value="//*[@id=\"row-2\"]/div/div/div/div[1]/div/div/article[{0}]/div[1]/div[3]/div/span/div/div/a".format(count))
        author = author_label.text
    except:
        author = ""
        header = ""
        desc = ""
            #print('Error 1')

    date_value = None
    try:
        date_element = driver.find_element(By.XPATH, value="/html[1]/body[1]/div[1]/main[1]/div[1]/section[1]/section[4]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/article[{0}]/div[1]/div[1]".format(count))

        if date_element.is_displayed():
            date_value = date_element.text
        else:
            date_element = driver.find_element(By.XPATH, value="/html[1]/body[1]/div[1]/main[1]/div[1]/section[1]/section[4]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/article[{0}]/div[1]/div[2]".format(count))
            date_value = date_element.text


    except:
        date_value=""
        #print('Error 2')

    if header != "":
        #print('store value {0}'.format(count))
        record = pd.DataFrame({
                                    'header':header.text,
                                    'desc': desc.text,
                                    'author': author,
                                    'date': date_value
                                }, index=[0])
        data = pd.concat([data, record],ignore_index=True, axis=0)
    count = count + 1
    
print('Completed load all articles.')

driver.quit()
data.reset_index()
#data.drop(columns=data.columns[0], axis=1, inplace=True)
data.to_csv(r"data/{0}_new_data.csv".format('forbes_money_news_raw'), sep="\t")
data

Item: 99re Button Progress: 47Completed load all articles.


,header,desc,author,date
0,Markets See Looming Recession – The Fed Sees “...,We suspect that the Fed will remain hawkish un...,Robert Barone,10 hours ago
1,Sixth Circuit Rules Low Income Housing Tax Cre...,It is almost inevitable that an offer that tri...,Peter J Reilly,10 hours ago
2,Sell? No Way - This Is A Buyers’ Stock Market,Don't miss out on this opportunity,John S. Tobey,11 hours ago
3,We Begin A Status Review Of The Over 200 Flori...,This Month We Begin A Status Review Of The Ove...,Richard Lehmann,12 hours ago
4,"To Avoid Falling For Scams, Novice Crypto Trad...",Some stocks are literally tokens off the block...,Rufas Kamau,21 hours ago
...,...,...,...,...
85,Delaware Chancery Refuses To Apply Ferengi Pri...,The written decision represents a scathing ack...,Peter J Reilly,"May 18, 2022"
86,Walmart and Target Miss Quarterly Earnings Est...,This was the backdrop of this week’s highly an...,Sanford Stein,"May 18, 2022"
87,Are We In A Bear Market? Here’s How To Tell,Between investor sentiment and economic factor...,Q.ai - Powering a Personal Wealth Movement,"May 18, 2022"
88,Clearlake Follows Its Chelsea Deal With A $14....,The Santa Monica-based firm has closed its big...,Kevin Dowd,"May 18, 2022"


# Data Processing
Manual process to clean up the data that extract from above script. We replace the data to a proper format such as 2 hours ago to specific date that extract on that day in order the data can be use in time series analysis later. 

In [7]:
import datetime as dt
from datetime import date
import numpy as np

news = pd.read_excel(r"Data/forbes_news/forbes_news.xlsx")

In [8]:
news

,number,label,header,desc,author,date
0,0,Stock Market,"Meet The Vermont Startup That Amazon, Gucci, A...","Superplastic, founded by Paul Budnitz in 2017,...",Marty Swant,2022-04-29
1,1,Stock Market,"Stock Market Fear Is Overdue, So Be Ready For ...",Wall Street is waiting for investor fear to sh...,John S. Tobey,2022-04-29
2,2,Finance,Getting A Handle On Inflation Affects How Far ...,"For Gen Xers, inflation can impact their retir...",Next Avenue,2022-04-29
3,3,Finance,After Widowhood Comes A Second Act With Purpose,What I learned after my husband’s death helped...,Next Avenue,2022-04-29
4,4,Finance,A Hopeful Future For Midlife Entrepreneurship,Read the inspiring stories of three mid-life e...,Next Avenue,2022-04-29
...,...,...,...,...,...,...
7525,7544,Finance,Thirty-Nine States Do Not Have Enough Money To...,39 states do not have enough money to\npay all...,Mayra Rodriguez Valladares,2021-10-01
7526,7545,Finance,"Hong Kong, Mainland China Bourses Close For Na...",Mainland exchanges shut until Oct. 8,Russell Flannery,2021-10-01
7527,7546,Stock Market,Apple Is Set-Up For Another Record Quarter,It appears Apple’s iPhone 13 is off to a stron...,Chuck Jones,2021-09-30
7528,7547,Finance,Charging Order Interstate Jurisdictional Issue...,A U.S. Magistrate Judge sitting in the Distric...,Jay Adkisson,2021-09-30


### Manual process to label news category.
Split the news into crypto, finance, stock market and metaverse category. Break down to specific news in order for us to find what news is more sensitive for price impact. 

In [10]:
start = dt.datetime(2021,9,30)
end = dt.datetime(2022,4,29)

def filter_news_by_date(start, end, data):
    data.date = pd.to_datetime(data.date)
    data = data[(data['date'] >= start) & (data['date'] <= end)]
    data = data.set_index('date') 
    data = data.sort_index()
    data = data.reset_index()
    #data = data.set_index('date')
    return data

In [11]:
news = pd.read_excel(r"Data/forbes_news/forbes_news_category.xlsx")
news = news.drop(['Column1'], axis=1)
news['category'] = np.where(news.category == 'Metaverse', 'Cryptocurrencies', news.category)
news_target_date = filter_news_by_date(start, end, news)
news_target_date

,date,category,content,sentiment_score
0,2021-09-30,Finance,got 000 student loan forgiveness forgiveness r...,-0.966413
1,2021-09-30,Finance,charging order interstate jurisdictional issue...,-0.480265
2,2021-09-30,Stock Market,apple set another record quarter appears apple...,0.971182
3,2021-10-01,Finance,hong kong mainland china bourses close nationa...,-0.966640
4,2021-10-01,Cryptocurrencies,cathie wood ark invest sold million tesla last...,0.037668
...,...,...,...,...
7525,2022-04-29,Finance,inflation peaked fed favorite indicator says m...,-0.570223
7526,2022-04-29,Finance,hong kong fintech unicorn looks raise million ...,0.802513
7527,2022-04-29,Stock Market,bearish sentiment jetblue airways stock revers...,0.704666
7528,2022-04-29,Finance,bernie sanders running president could mean st...,-0.892253


In [12]:
news_target_date.groupby(['category']).size().reset_index(name='counts')

,category,counts
0,Cryptocurrencies,960
1,Finance,4715
2,Stock Market,1855
